In [10]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
#import project modules
from modules import *

In [11]:
def rand_arr(a, b, *args): 
    np.random.seed(0)
    return np.random.rand(*args) * (b - a) + a

In [12]:
class LstmParam:
    def __init__(self, ct_dim, x_dim):
        """
        
        initialize all weights and biases 
        ct_dim - the dimension of current cell state (Ct) matrix
        x_dim - the dimension of Tth input
        
        Weight and bias matrices are initialized with random values instead of zeroes to add noise. 
        Their derivatives will thus be zero.
        
        Terminology:
        
        Prefixes
        w - a weight matrix
        b - a bias matrix
        d - a derivative matrix
        
        Suffixes
        c - cell state gate. Represents data held in current cell. 
            It is the c't (c bar t) matrix
        i - input gate
        f - forget gate
        o - output gate        
        
        """
        self.ct_dim = ct_dim  
        self.x_dim = x_dim 
        combined_dim = x_dim + ct_dim
        
        # Initialize weights
        self.wc = rand_arr(-0.1, 0.1, ct_dim, combined_dim)
        self.wi = rand_arr(-0.1, 0.1, ct_dim, combined_dim) 
        self.wf = rand_arr(-0.1, 0.1, ct_dim, combined_dim)
        self.wo = rand_arr(-0.1, 0.1, ct_dim, combined_dim)
        
        #Initialize biases 
        self.bc = rand_arr(-0.1, 0.1, ct_dim) 
        self.bi = rand_arr(-0.1, 0.1, ct_dim) 
        self.bf = rand_arr(-0.1, 0.1, ct_dim) 
        self.bo = rand_arr(-0.1, 0.1, ct_dim) 
        
        # Initialize derivatives
        self.dwc = np.zeros((ct_dim, combined_dim)) 
        self.dwi = np.zeros((ct_dim, combined_dim)) 
        self.dwf = np.zeros((ct_dim, combined_dim)) 
        self.dwo = np.zeros((ct_dim, combined_dim)) 
        self.dbc = np.zeros(ct_dim) 
        self.dbi = np.zeros(ct_dim) 
        self.dbf = np.zeros(ct_dim) 
        self.dbo = np.zeros(ct_dim)
        
    def apply_derivatives(self, alpha = 1):
        """
        Update parameters in each iteration to reach optimal value 
        alpha is the learning rate.
        """
        self.wc -= alpha * self.dwc
        self.wi -= alpha * self.dwi
        self.wf -= alpha * self.dwf
        self.wo -= alpha * self.dwo
        self.bc -= alpha * self.dbc
        self.bi -= alpha * self.dbi
        self.bf -= alpha * self.dbf
        self.bo -= alpha * self.dbo
        
        # reset all derivatives to zero
        self.dwc = np.zeros_like(self.wc)
        self.dwi = np.zeros_like(self.wi) 
        self.dwf = np.zeros_like(self.wf) 
        self.dwo = np.zeros_like(self.wo) 
        self.dbc = np.zeros_like(self.bc)
        self.dbi = np.zeros_like(self.bi) 
        self.dbf = np.zeros_like(self.bf) 
        self.dbo = np.zeros_like(self.bo) 

In [13]:
class LstmCellState:
    def __init__(self, ct_dim, x_dim):
        """
        Initialize all gate matrices. 
        All gate matrices have the same dimension as ct matrix
        c - current hidden cell state. The gate corresponding to this determines how much 
            data of previous cell should be read in current cell.
        i - input gate. Determines how much data should be read into the cell from current input.
        f - forget gate. Determines how much data should be forgotten, i.e discarded
        o - output gate. How much data to output from current cell to next cell
        s - The present state of gate. 
            Equation to calculate present state : forget_gate*previous_state(s<t-1>) + c_gate*input_gate
        h - output state of the cell. It is the prediction value of Tth output in series.
        dh - derivative of h
        ds - derivative of s
        """
        self.c = np.zeros(ct_dim)
        self.i = np.zeros(ct_dim)
        self.f = np.zeros(ct_dim)
        self.o = np.zeros(ct_dim)
        self.s = np.zeros(ct_dim)
        self.h = np.zeros(ct_dim)
        self.dh = np.zeros_like(self.h)
        self.ds = np.zeros_like(self.s)

In [14]:
class LstmLayer:
    def __init__(self, lstm_param, lstm_cell_state):
        self.state = lstm_cell_state
        self.param = lstm_param
        #xh is the concatenation ofprevious layer's output with current input.
        self.xh = None

    def calculate_gates(self, x, s_prev = None, h_prev = None):
        """
        if this is the first lstm layer in the network then 
        s_prev and h_prev will be initialized to zero as cell state 
        and output states are not present.
        
        s_prev = cell state of previous layer's cells.
        h_prev = output state from previous layer
        
        """
        if s_prev is None: 
            s_prev = np.zeros_like(self.state.s)
        if h_prev is None: 
            h_prev = np.zeros_like(self.state.h)
        
        # save previous states for use in backprop
        self.s_prev = s_prev
        self.h_prev = h_prev

        # concatenate x(T) and h(T-1)
        xh = np.hstack((x,  h_prev))
        #Calculate gate values
        self.state.c = tanh(np.dot(self.param.wc, xh) + self.param.bc)
        self.state.i = sigmoid(np.dot(self.param.wi, xh) + self.param.bi)
        self.state.f = sigmoid(np.dot(self.param.wf, xh) + self.param.bf)
        self.state.o = sigmoid(np.dot(self.param.wo, xh) + self.param.bo)
        self.state.s = self.state.c * self.state.i + s_prev * self.state.f
        self.state.h = self.state.s * self.state.o
        self.xh = xh
    
    def calculate_derivatives(self, dh, ds):
        ds = self.state.o * dh + ds
        do = self.state.s * dh
        di = self.state.c * ds
        dc = self.state.i * ds
        df = self.s_prev * ds

        # calculate derivatives w.r.t. gate inside sigma / tanh function
        di_input = dsigmoid(self.state.i) * di 
        df_input = dsigmoid(self.state.f) * df 
        do_input = dsigmoid(self.state.o) * do 
        dc_input = dtanh(self.state.c) * dc

        # derivatives w.r.t. inputs
        self.param.dwc += np.outer(dc_input, self.xh)
        self.param.dwi += np.outer(di_input, self.xh)
        self.param.dwf += np.outer(df_input, self.xh)
        self.param.dwo += np.outer(do_input, self.xh)
        self.param.dbc += dc_input       
        self.param.dbi += di_input
        self.param.dbf += df_input       
        self.param.dbo += do_input
        
        # calculate derivative for xh
        dxh = np.zeros_like(self.xh)
        dxh += np.dot(self.param.wc.T, dc_input)
        dxh += np.dot(self.param.wi.T, di_input)
        dxh += np.dot(self.param.wf.T, df_input)
        dxh += np.dot(self.param.wo.T, do_input)
        
        # save derivatives
        self.state.ds = ds * self.state.f
        self.state.dh = dxh[self.param.x_dim:]

In [15]:
class LstmNetwork():
    def __init__(self, lstm_param):
        """
        x_list - the sequence that'll be iput to lstm
        lstm_layer_list - the ouput from layer that will be input to next layer
        """
        self.lstm_param = lstm_param
        self.lstm_layer_list = []
        self.x_list = []

    def get_loss(self, y_list, loss_layer):
        """
        Updates derivatives w.r.t corresponding loss layer. 
        To update parameters, we will call self.lstm_param.apply_derivatives()
        """
        assert len(y_list) == len(self.x_list)
        index = len(self.x_list) - 1
        # Calculate loss for the last layer 
        loss = loss_layer.loss(self.lstm_layer_list[index].state.h, y_list[index])
        diff_h = loss_layer.derivative(self.lstm_layer_list[index].state.h, y_list[index])
        # For the last layer of the network, diff_s will be
        diff_s = np.zeros(self.lstm_param.ct_dim)
        self.lstm_layer_list[index].calculate_derivatives(diff_h, diff_s)
        index -= 1

        while index >= 0:
            loss += loss_layer.loss(self.lstm_layer_list[index].state.h, y_list[index])
            diff_h = loss_layer.derivative(self.lstm_layer_list[index].state.h, y_list[index])
            diff_h += self.lstm_layer_list[index + 1].state.dh
            diff_s = self.lstm_layer_list[index + 1].state.ds
            self.lstm_layer_list[index].calculate_derivatives(diff_h, diff_s)
            index -= 1 

        return loss

    def clear_x_list(self):
        self.x_list = []

    def add_x_list(self, x):
        self.x_list.append(x)
        if len(self.x_list) > len(self.lstm_layer_list):
            lstm_state = LstmCellState(self.lstm_param.ct_dim, self.lstm_param.x_dim)
            self.lstm_layer_list.append(LstmLayer(self.lstm_param, lstm_state))

        # get index of most recent x input
        index = len(self.x_list) - 1
        if index == 0:
            self.lstm_layer_list[index].calculate_gates(x)
        else:
            s_prev = self.lstm_layer_list[index - 1].state.s
            h_prev = self.lstm_layer_list[index - 1].state.h
            self.lstm_layer_list[index].calculate_gates(x, s_prev, h_prev)

In [18]:
def normalize_dataset(stock_market_dataset):
    normalizer = preprocessing.MinMaxScaler((0,1))
    smd_exclude_strings = stock_market_dataset.select_dtypes(include = [np.number])
    normalized = normalizer.fit_transform(smd_exclude_strings)
    #normalized = normalizer.inverse_transform(normalized)
    return [normalized, normalizer]

def denormalize_data(df, normalizer, time_step, predictions, close_value_col_index):
    df_copy = df.copy(deep = True)
    rows_to_drop = [i for i in range(0,time_step)]
    df_copy.drop(rows_to_drop,inplace = True)     
    #deleting first #time_step rows of normalized_df. Because output prediction for the first 30 days (first time slice) doesn't exist.

    #converting prediction array to dataframe for replacing actual row with prediction row
    predicted_normalised_values_df = pd.DataFrame(predictions) 

    #replacing
    df_copy[close_value_col_index] = predicted_normalised_values_df[0].values
    
    temp = normalizer.inverse_transform(df_copy)
    df_copy = pd.DataFrame(temp)
    return df_copy

def make_train_test_val_sets(dataarray, num_features, close_value_col_index, time_step, train_per, val_per, test_per):
    n = dataarray.shape[0] 

    #slices of data into time_steps 
    X_slice=[]
    y_slice=[]
    #normalized df dim 3125, 13
    # i = 30 - 3125

    for i in range(time_step, n):
        X_slice.append(dataarray[ i-time_step:i ,  close_value_col_index] )      #1 example having data from 30 days dim 30x13
        y_slice.append(dataarray[i,close_value_col_index])                  #close value data of the 31st day 

    #splitting percentage

    #splitting slices for test,val,train and converting into np array
    X_train=np.array( X_slice[ 0:int(n*train_per) ])
    y_train=np.array( y_slice[ 0:int(n*train_per) ])

    X_val=np.array( X_slice[ int(n*train_per):int(n*(train_per+val_per)) ])
    y_val=np.array( y_slice[ int(n*train_per):int(n*(train_per+val_per)) ])

    X_test=np.array( X_slice[ int(n*(train_per+val_per)): ])
    y_test=np.array( y_slice[ int(n*(train_per+val_per)): ])
    
    print(np.array(X_train).shape)
    #X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], num_features))    
    #dimension of X_train 3095, 30, 13
    #dimension of y_train  
    print(np.array(X_train).shape)
    return [X_train, y_train, X_val, y_val, X_test, y_test]

In [27]:
data_set_file = 'Google.csv'
stock_market_dataset = pd.read_csv(data_set_file)
stock_market_dataset.shape
stock_market_dataset.head(3125)
stock_market_dataset.columns.tolist()

###normalize data
normalized_array, normalizer = normalize_dataset(stock_market_dataset)
#normalized_df.head()
#n = normalized_df.shape[0]

###split data set into train, cross validation and test set

num_features = 6 
close_value_col_index = 3
#creating training set with time steps.

time_step=30    #1 month time step
train_per = 0.01
val_per = 0.2
test_per = 1-(train_per+val_per)  #0.1
    
X_train, y_train, X_val, y_val, X_test, y_test =  make_train_test_val_sets(normalized_array, num_features, close_value_col_index, time_step, train_per, val_per, test_per)

print(str(X_train.shape)+ " training dataset shape")
print(str(X_val.shape)+ "  val dataset shape")
print(str(X_test.shape)+ "  test dataset shape")
for i in range(31):
    print(X_train[i],y_train[i])

(31, 30)
(31, 30)
(31, 30) training dataset shape
(625, 30)  val dataset shape
(2439, 30)  test dataset shape
[0.00029014 0.00740966 0.00838273 0.00433867 0.00534745 0.00705256
 0.00548136 0.00178546 0.00210684 0.00021426 0.00133909 0.
 0.00140159 0.00204435 0.00205328 0.00474932 0.00668654 0.01024854
 0.01070383 0.01246251 0.01560491 0.01727432 0.01591737 0.01639944
 0.0185777  0.0176939  0.01629232 0.02396979 0.02773711 0.02641587] 0.029076203399514347
[0.00740966 0.00838273 0.00433867 0.00534745 0.00705256 0.00548136
 0.00178546 0.00210684 0.00021426 0.00133909 0.         0.00140159
 0.00204435 0.00205328 0.00474932 0.00668654 0.01024854 0.01070383
 0.01246251 0.01560491 0.01727432 0.01591737 0.01639944 0.0185777
 0.0176939  0.01629232 0.02396979 0.02773711 0.02641587 0.0290762 ] 0.0312901728324525
[0.00838273 0.00433867 0.00534745 0.00705256 0.00548136 0.00178546
 0.00210684 0.00021426 0.00133909 0.         0.00140159 0.00204435
 0.00205328 0.00474932 0.00668654 0.01024854 0.010703

In [34]:
class LossLayer:
    """
    Computes square loss with first element of hidden layer array.
    """
    @classmethod
    def loss(self, pred, label):
        return (pred[0] - label) ** 2

    @classmethod
    def derivative(self, pred, label):
        diff = np.zeros_like(pred)
        diff[0] = 2 * (pred[0] - label)
        return diff


def example_0():
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    # parameters for input data dimension and lstm cell count
    mem_cell_ct = 100
    x_dim = 30
    lstm_param = LstmParam(mem_cell_ct, x_dim)
    lstm_net = LstmNetwork(lstm_param)
    #y_list = [-0.5, 0.2, 0.1, -0.5]
    y_list = y_train.tolist()
    #input_val_arr = [np.random.random(x_dim) for _ in y_list]
    input_val_arr = X_train.tolist()

    for cur_iter in range(1000):
        print("iter", "%2s" % str(cur_iter), end=": ")
        for ind in range(len(y_list)):
            lstm_net.add_x_list(input_val_arr[ind])

        print("y_pred = [" [in+
              ", ".join(["[% 2.5f % 2.5f]" % (lstm_net.lstm_layer_list[ind].state.h[0], y_listd]) for ind in range(len(y_list))]) +
              "]", end=", ")

        loss = lstm_net.get_loss(y_list, LossLayer)
        print("loss:", "%.3e" % loss)
        lstm_param.iapply_dervatives(0.1)
        lstm_net.clear_x_list()



example_0()

iter  0: y_pred = [[ 0.00380  0.02908], [ 0.00595  0.03129], [ 0.00687  0.03425], [ 0.00714  0.03309], [ 0.00721  0.03467], [ 0.00719  0.03367], [ 0.00706  0.03147], [ 0.00749  0.03338], [ 0.00789  0.03650], [ 0.00836  0.03749], [ 0.00902  0.03937], [ 0.00959  0.04388], [ 0.00982  0.04279], [ 0.00991  0.03614], [ 0.01022  0.04407], [ 0.01030  0.06465], [ 0.00992  0.07802], [ 0.01005  0.07302], [ 0.01044  0.07674], [ 0.01099  0.08328], [ 0.01119  0.08091], [ 0.01145  0.08572], [ 0.01149  0.08468], [ 0.01146  0.08183], [ 0.01199  0.07561], [ 0.01287  0.06190], [ 0.01412  0.06476], [ 0.01537  0.06132], [ 0.01649  0.06057], [ 0.01714  0.07411], [ 0.01639  0.07319]], loss: 7.352e-02
iter  1: y_pred = [[ 0.03937  0.02908], [ 0.06205  0.03129], [ 0.07481  0.03425], [ 0.08193  0.03309], [ 0.08600  0.03467], [ 0.08832  0.03367], [ 0.08956  0.03147], [ 0.09098  0.03338], [ 0.09198  0.03650], [ 0.09290  0.03749], [ 0.09397  0.03937], [ 0.09483  0.04388], [ 0.09524  0.04279], [ 0.09547  0.03614], 

iter 12: y_pred = [[ 0.02253  0.02908], [ 0.03525  0.03129], [ 0.04219  0.03425], [ 0.04594  0.03309], [ 0.04808  0.03467], [ 0.04930  0.03367], [ 0.04993  0.03147], [ 0.05093  0.03338], [ 0.05171  0.03650], [ 0.05248  0.03749], [ 0.05343  0.03937], [ 0.05423  0.04388], [ 0.05465  0.04279], [ 0.05491  0.03614], [ 0.05540  0.04407], [ 0.05562  0.06465], [ 0.05539  0.07802], [ 0.05583  0.07302], [ 0.05656  0.07674], [ 0.05747  0.08328], [ 0.05798  0.08091], [ 0.05857  0.08572], [ 0.05891  0.08468], [ 0.05915  0.08183], [ 0.06004  0.07561], [ 0.06124  0.06190], [ 0.06280  0.06476], [ 0.06431  0.06132], [ 0.06563  0.06057], [ 0.06640  0.07411], [ 0.06561  0.07319]], loss: 7.544e-03
iter 13: y_pred = [[ 0.02356  0.02908], [ 0.03691  0.03129], [ 0.04423  0.03425], [ 0.04822  0.03309], [ 0.05050  0.03467], [ 0.05182  0.03367], [ 0.05251  0.03147], [ 0.05355  0.03338], [ 0.05436  0.03650], [ 0.05515  0.03749], [ 0.05612  0.03937], [ 0.05694  0.04388], [ 0.05737  0.04279], [ 0.05765  0.03614], 

iter 23: y_pred = [[ 0.02193  0.02908], [ 0.03462  0.03129], [ 0.04177  0.03425], [ 0.04579  0.03309], [ 0.04818  0.03467], [ 0.04962  0.03367], [ 0.05043  0.03147], [ 0.05157  0.03338], [ 0.05246  0.03650], [ 0.05334  0.03749], [ 0.05439  0.03937], [ 0.05529  0.04388], [ 0.05581  0.04279], [ 0.05617  0.03614], [ 0.05675  0.04407], [ 0.05704  0.06465], [ 0.05694  0.07802], [ 0.05754  0.07302], [ 0.05844  0.07674], [ 0.05951  0.08328], [ 0.06020  0.08091], [ 0.06095  0.08572], [ 0.06145  0.08468], [ 0.06185  0.08183], [ 0.06288  0.07561], [ 0.06420  0.06190], [ 0.06585  0.06476], [ 0.06744  0.06132], [ 0.06882  0.06057], [ 0.06965  0.07411], [ 0.06892  0.07319]], loss: 6.936e-03
iter 24: y_pred = [[ 0.02177  0.02908], [ 0.03440  0.03129], [ 0.04153  0.03425], [ 0.04555  0.03309], [ 0.04795  0.03467], [ 0.04940  0.03367], [ 0.05022  0.03147], [ 0.05137  0.03338], [ 0.05227  0.03650], [ 0.05315  0.03749], [ 0.05421  0.03937], [ 0.05512  0.04388], [ 0.05564  0.04279], [ 0.05601  0.03614], 

iter 35: y_pred = [[ 0.02070  0.02908], [ 0.03297  0.03129], [ 0.04007  0.03425], [ 0.04420  0.03309], [ 0.04674  0.03467], [ 0.04834  0.03367], [ 0.04929  0.03147], [ 0.05054  0.03338], [ 0.05154  0.03650], [ 0.05251  0.03749], [ 0.05365  0.03937], [ 0.05464  0.04388], [ 0.05525  0.04279], [ 0.05570  0.03614], [ 0.05635  0.04407], [ 0.05673  0.06465], [ 0.05674  0.07802], [ 0.05750  0.07302], [ 0.05856  0.07674], [ 0.05980  0.08328], [ 0.06065  0.08091], [ 0.06155  0.08572], [ 0.06221  0.08468], [ 0.06275  0.08183], [ 0.06390  0.07561], [ 0.06533  0.06190], [ 0.06704  0.06476], [ 0.06870  0.06132], [ 0.07014  0.06057], [ 0.07101  0.07411], [ 0.07034  0.07319]], loss: 6.483e-03
iter 36: y_pred = [[ 0.02061  0.02908], [ 0.03284  0.03129], [ 0.03994  0.03425], [ 0.04407  0.03309], [ 0.04663  0.03467], [ 0.04824  0.03367], [ 0.04920  0.03147], [ 0.05046  0.03338], [ 0.05146  0.03650], [ 0.05244  0.03749], [ 0.05359  0.03937], [ 0.05459  0.04388], [ 0.05521  0.04279], [ 0.05566  0.03614], 

iter 50: y_pred = [[ 0.01948  0.02908], [ 0.03130  0.03129], [ 0.03833  0.03425], [ 0.04256  0.03309], [ 0.04526  0.03467], [ 0.04702  0.03367], [ 0.04811  0.03147], [ 0.04949  0.03338], [ 0.05059  0.03650], [ 0.05166  0.03749], [ 0.05290  0.03937], [ 0.05398  0.04388], [ 0.05470  0.04279], [ 0.05525  0.03614], [ 0.05599  0.04407], [ 0.05646  0.06465], [ 0.05660  0.07802], [ 0.05753  0.07302], [ 0.05877  0.07674], [ 0.06019  0.08328], [ 0.06122  0.08091], [ 0.06229  0.08572], [ 0.06310  0.08468], [ 0.06379  0.08183], [ 0.06507  0.07561], [ 0.06661  0.06190], [ 0.06840  0.06476], [ 0.07011  0.06132], [ 0.07160  0.06057], [ 0.07252  0.07411], [ 0.07193  0.07319]], loss: 6.058e-03
iter 51: y_pred = [[ 0.01941  0.02908], [ 0.03120  0.03129], [ 0.03823  0.03425], [ 0.04246  0.03309], [ 0.04517  0.03467], [ 0.04694  0.03367], [ 0.04804  0.03147], [ 0.04942  0.03338], [ 0.05053  0.03650], [ 0.05161  0.03749], [ 0.05285  0.03937], [ 0.05394  0.04388], [ 0.05467  0.04279], [ 0.05522  0.03614], 

...

iter 932: y_pred = [[ 0.02487  0.02908], [ 0.03260  0.03129], [ 0.03505  0.03425], [ 0.03644  0.03309], [ 0.03774  0.03467], [ 0.03908  0.03367], [ 0.04009  0.03147], [ 0.04139  0.03338], [ 0.04258  0.03650], [ 0.04403  0.03749], [ 0.04584  0.03937], [ 0.04773  0.04388], [ 0.04960  0.04279], [ 0.05126  0.03614], [ 0.05258  0.04407], [ 0.05391  0.06465], [ 0.05619  0.07802], [ 0.06027  0.07302], [ 0.06412  0.07674], [ 0.06755  0.08328], [ 0.07013  0.08091], [ 0.07188  0.08572], [ 0.07292  0.08468], [ 0.07313  0.08183], [ 0.07325  0.07561], [ 0.07303  0.06190], [ 0.07229  0.06476], [ 0.07194  0.06132], [ 0.07204  0.06057], [ 0.07250  0.07411], [ 0.07322  0.07319]], loss: 2.807e-03
iter 933: y_pred = [[ 0.02491  0.02908], [ 0.03266  0.03129], [ 0.03513  0.03425], [ 0.03654  0.03309], [ 0.03786  0.03467], [ 0.03920  0.03367], [ 0.04022  0.03147], [ 0.04152  0.03338], [ 0.04272  0.03650], [ 0.04418  0.03749], [ 0.04599  0.03937], [ 0.04788  0.04388], [ 0.04976  0.04279], [ 0.05141  0.03614]

iter 948: y_pred = [[ 0.02444  0.02908], [ 0.03203  0.03129], [ 0.03450  0.03425], [ 0.03595  0.03309], [ 0.03734  0.03467], [ 0.03875  0.03367], [ 0.03983  0.03147], [ 0.04118  0.03338], [ 0.04241  0.03650], [ 0.04390  0.03749], [ 0.04575  0.03937], [ 0.04768  0.04388], [ 0.04958  0.04279], [ 0.05127  0.03614], [ 0.05262  0.04407], [ 0.05398  0.06465], [ 0.05631  0.07802], [ 0.06045  0.07302], [ 0.06437  0.07674], [ 0.06786  0.08328], [ 0.07049  0.08091], [ 0.07227  0.08572], [ 0.07333  0.08468], [ 0.07355  0.08183], [ 0.07367  0.07561], [ 0.07343  0.06190], [ 0.07267  0.06476], [ 0.07231  0.06132], [ 0.07240  0.06057], [ 0.07287  0.07411], [ 0.07362  0.07319]], loss: 2.755e-03
iter 949: y_pred = [[ 0.02442  0.02908], [ 0.03201  0.03129], [ 0.03447  0.03425], [ 0.03593  0.03309], [ 0.03732  0.03467], [ 0.03874  0.03367], [ 0.03982  0.03147], [ 0.04117  0.03338], [ 0.04241  0.03650], [ 0.04390  0.03749], [ 0.04575  0.03937], [ 0.04768  0.04388], [ 0.04959  0.04279], [ 0.05128  0.03614]

iter 965: y_pred = [[ 0.02407  0.02908], [ 0.03154  0.03129], [ 0.03398  0.03425], [ 0.03548  0.03309], [ 0.03694  0.03467], [ 0.03841  0.03367], [ 0.03954  0.03147], [ 0.04093  0.03338], [ 0.04220  0.03650], [ 0.04372  0.03749], [ 0.04560  0.03937], [ 0.04756  0.04388], [ 0.04950  0.04279], [ 0.05122  0.03614], [ 0.05259  0.04407], [ 0.05398  0.06465], [ 0.05636  0.07802], [ 0.06057  0.07302], [ 0.06454  0.07674], [ 0.06808  0.08328], [ 0.07074  0.08091], [ 0.07255  0.08572], [ 0.07363  0.08468], [ 0.07385  0.08183], [ 0.07396  0.07561], [ 0.07371  0.06190], [ 0.07291  0.06476], [ 0.07253  0.06132], [ 0.07261  0.06057], [ 0.07307  0.07411], [ 0.07385  0.07319]], loss: 2.711e-03
iter 966: y_pred = [[ 0.02405  0.02908], [ 0.03151  0.03129], [ 0.03396  0.03425], [ 0.03546  0.03309], [ 0.03691  0.03467], [ 0.03840  0.03367], [ 0.03953  0.03147], [ 0.04092  0.03338], [ 0.04219  0.03650], [ 0.04371  0.03749], [ 0.04559  0.03937], [ 0.04755  0.04388], [ 0.04950  0.04279], [ 0.05122  0.03614]

iter 982: y_pred = [[ 0.02381  0.02908], [ 0.03116  0.03129], [ 0.03358  0.03425], [ 0.03511  0.03309], [ 0.03661  0.03467], [ 0.03814  0.03367], [ 0.03931  0.03147], [ 0.04073  0.03338], [ 0.04202  0.03650], [ 0.04357  0.03749], [ 0.04547  0.03937], [ 0.04746  0.04388], [ 0.04943  0.04279], [ 0.05118  0.03614], [ 0.05257  0.04407], [ 0.05398  0.06465], [ 0.05640  0.07802], [ 0.06067  0.07302], [ 0.06469  0.07674], [ 0.06827  0.08328], [ 0.07097  0.08091], [ 0.07279  0.08572], [ 0.07387  0.08468], [ 0.07410  0.08183], [ 0.07418  0.07561], [ 0.07391  0.06190], [ 0.07307  0.06476], [ 0.07266  0.06132], [ 0.07273  0.06057], [ 0.07320  0.07411], [ 0.07401  0.07319]], loss: 2.672e-03
iter 983: y_pred = [[ 0.02379  0.02908], [ 0.03114  0.03129], [ 0.03356  0.03425], [ 0.03509  0.03309], [ 0.03659  0.03467], [ 0.03812  0.03367], [ 0.03929  0.03147], [ 0.04072  0.03338], [ 0.04202  0.03650], [ 0.04356  0.03749], [ 0.04547  0.03937], [ 0.04746  0.04388], [ 0.04943  0.04279], [ 0.05118  0.03614]

iter 998: y_pred = [[ 0.02363  0.02908], [ 0.03089  0.03129], [ 0.03329  0.03425], [ 0.03483  0.03309], [ 0.03636  0.03467], [ 0.03793  0.03367], [ 0.03912  0.03147], [ 0.04057  0.03338], [ 0.04189  0.03650], [ 0.04345  0.03749], [ 0.04538  0.03937], [ 0.04739  0.04388], [ 0.04938  0.04279], [ 0.05116  0.03614], [ 0.05255  0.04407], [ 0.05398  0.06465], [ 0.05644  0.07802], [ 0.06076  0.07302], [ 0.06482  0.07674], [ 0.06843  0.08328], [ 0.07116  0.08091], [ 0.07299  0.08572], [ 0.07408  0.08468], [ 0.07429  0.08183], [ 0.07436  0.07561], [ 0.07405  0.06190], [ 0.07318  0.06476], [ 0.07274  0.06132], [ 0.07279  0.06057], [ 0.07326  0.07411], [ 0.07409  0.07319]], loss: 2.639e-03
iter 999: y_pred = [[ 0.02362  0.02908], [ 0.03088  0.03129], [ 0.03327  0.03425], [ 0.03482  0.03309], [ 0.03635  0.03467], [ 0.03791  0.03367], [ 0.03911  0.03147], [ 0.04056  0.03338], [ 0.04188  0.03650], [ 0.04345  0.03749], [ 0.04537  0.03937], [ 0.04738  0.04388], [ 0.04938  0.04279], [ 0.05115  0.03614]